In [1]:
import pandas as pd
import numpy as np
import cobra
import re
from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import CustomSelectionIntegrationStrategy
from troppo.methods.reconstruction.fastcore import FASTcore, FastcoreProperties
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.gapfill.fastcc import FastCC, FastCCProperties
from cobra.io import read_sbml_model
import numpy as np

# parsing rule 
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')

expression_data = pd.read_csv('/home/biodata/aman/data/data_processed/expression_data_hippocampus-development-human-brain-10XV2_annotated_normalized_annotated_gencode.csv', index_col=0)
omics_container = TabularReader(path_or_df=expression_data, nomenclature='ensembl_gene_id',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)
from math import log

threshold =  (5 * log(2))
protected_reactions = []

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > threshold) or k in protected_reactions]]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])
threshold_scores = threshold_integration.integrate(data_map=data_map)
print(threshold_scores) # why is this empty
print(data_map.get_scores())

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/wrappers/external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000147576 or _ENSG00000172955 or _ENSG00000180011 or _ENSG00000187758 or _ENSG00000196344 or _ENSG00000196616 or _ENSG00000197894 or _ENSG00000198099 or _ENSG00000248144
  warnings.warn('Could not normalize 

[[]]
{'MAR03905': None, 'MAR03907': None, 'MAR04097': None, 'MAR04099': None, 'MAR04108': None, 'MAR04133': None, 'MAR04281': None, 'MAR04388': None, 'MAR04283': None, 'MAR08357': None, 'MAR04379': None, 'MAR04301': None, 'MAR04355': None, 'MAR04358': None, 'MAR04360': None, 'MAR04363': None, 'MAR04365': None, 'MAR04368': None, 'MAR04370': None, 'MAR04371': None, 'MAR04372': None, 'MAR04373': None, 'MAR04375': None, 'MAR04377': None, 'MAR04381': None, 'MAR04391': None, 'MAR04394': None, 'MAR04396': None, 'MAR04521': None, 'MAR06412': None, 'MAR07745': None, 'MAR07747': None, 'MAR08360': None, 'MAR08652': None, 'MAR08757': None, 'MAR03989': None, 'MAR04122': None, 'MAR04837': None, 'MAR05395': None, 'MAR05396': None, 'MAR09727': None, 'MAR05397': None, 'MAR05398': None, 'MAR05399': None, 'MAR05400': None, 'MAR05401': None, 'MAR08568': None, 'MAR08569': None, 'MAR08570': None, 'MAR08571': None, 'MAR08572': None, 'MAR08573': None, 'MAR08574': None, 'MAR08575': None, 'MAR08576': None, 'MAR

In [11]:
from math import log

threshold =  (0.2 * log(2))

protected_reactions = ["MAR04413", "MAR09931", "MAR09932","MAR10024","MAR10062", "MAR10063", "MAR10064", "MAR10065","MAR13082"]

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > threshold) or k in protected_reactions]]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])

threshold_scores = threshold_integration.integrate(data_map=data_map)

print(threshold_scores)
print([len(i) for i in threshold_scores])

[['MAR04413', 'MAR09931', 'MAR09932', 'MAR10024', 'MAR10062', 'MAR10063', 'MAR10064', 'MAR10065', 'MAR13082']]
[9]


In [12]:
# Get the index of the reaction of the CORE reaction set
ordered_ids = {r:i for i,r in enumerate(model_wrapper.model_reader.r_ids)}
core_idx = [[ordered_ids[k] for k in l] for l in threshold_scores]
print([len(i) for i in core_idx])

[9]


In [21]:
properties = FastCCProperties(
    flux_threshold = 1e-4,
    method = "original",
    solver = "CPLEX"
)

fastcc = FastCC(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)
result_reactions = fastcc.run()

print(len(result_reactions))

[]
11694 Total reactions
[11694 11693 11692 ...     6     5     4] Reversible reactions
6307 Irreversible reactions
LP7
-0.6306999999999469
11275 Flux consistent reactions, without flipping
419 Flux inconsistent reactions, without flipping
LP7
-0.03539999999999999
11667 Flux consistent reactions
27 Flux inconsistent reversible reactions left to flip
LP7
0.0
11671 Flux consistent reactions
23 Flux inconsistent reversible reactions left to flip
LP7
0.0
11673 Flux consistent reactions
21 Flux inconsistent reversible reactions left to flip
LP7
0.0
21 reversible reactions flipped
LP7
-0.00030000000000000003
sanity check
11680 Flux consistent reactions
14 Flux inconsistent reversible reactions left to flip
LP7
0.0
sanity check
11682 Flux consistent reactions
12 Flux inconsistent reversible reactions left to flip
LP7
0.0
sanity check
11684 Flux consistent reactions
10 Flux inconsistent reversible reactions left to flip
LP7
0.0
10 reversible reactions flipped
LP7
0.0
sanity check
11686 Flux co

In [25]:
# Define the FastCORE properties
properties = FastcoreProperties(core=core_idx, solver='CPLEX')
# instantiate the FastCORE class
fastcore = FASTcore(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)
# Run the algorithm
model_fastcore = fastcore.run()

J size4866
[    0     1     2 ... 11688 11689 11690]
before LP7
LP7
-0.48659999999996273
done LP7
LP9
54625081.347482964
Warning, Solution is not optimal
done LP9
449 8483
before LP7
LP7
-0.04080000000000014
done LP7
LP9
6175.000000011409
Warning, Solution is not optimal
done LP9
449 9519


ValueError: operands could not be broadcast together with shapes (426,) (0,) 

In [22]:
print(pd.DataFrame(model_fastcore))

NameError: name 'model_fastcore' is not defined

In [ ]:
print(type(model_fastcore))

<class 'list'>


In [ ]:
pd.DataFrame(model_fastcore).to_csv('fastcore_output_tinit.tsv', sep='\t')

---

In [ ]:
selected_reactions = [model.reactions[i] for i in model_fastcore]

print(selected_reactions)

[]

In [ ]:
ctx_model = model.copy()
selected_ids = [r.id for r in selected_reactions]
to_remove = [r for r in ctx_model.reactions if r.id not in selected_ids]
ctx_model.remove_reactions(to_remove, remove_orphans=True)
ctx_model

/home/biodata/miniconda3/envs/cplex_aman/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Name,HumanGEM
Memory address,7ac63efd7bb0
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,148
Objective expression,0
Compartments,


In [ ]:
print(len(model.reactions), len(ctx_model.reactions))

(12971, 0)

In [ ]:
# export
cobra.io.write_sbml_model(ctx_model, "fastcore_context_specific_model.xml")

In [ ]:
solution = ctx_model.optimize()
print(solution.objective_value)

0.0


In [ ]:
print(ctx_model.summary())

Non-linear or non-reaction model objective. Falling back to minimal display.


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux
